In [1]:
dataset_name = "W19_comb"
df_list = [ "BES_Panel", "BES_reduced_with_na"]

In [2]:
import warnings
warnings.filterwarnings('ignore')

from IPython.display import display
from IPython.display import HTML
import IPython.core.display as di # Example: di.display_html('<h3>%s:</h3>' % str, raw=True)

# This line will hide code by default when the notebook is exported as HTML
di.display_html('<script>jQuery(function() {if (jQuery("body.notebook_app").length == 0) { jQuery(".input_area").toggle(); jQuery(".prompt").toggle();}});</script>', raw=True)

# This line will add a button to toggle visibility of code blocks, for use with the HTML export version
di.display_html('''<button onclick="jQuery('.input_area').toggle(); jQuery('.prompt').toggle();">Toggle code</button>''', raw=True)

%matplotlib inline

import numpy as np, pandas as pd, matplotlib.pyplot as plt, seaborn as sns
import pickle, os, gc, re

sns.set();
sns.set_palette("colorblind")

from IPython.display import display, display_html, HTML
from IPython.core.debugger import set_trace
# plt.rcParams["axes.grid"] = False

import Jupyter_module_loader
from utility import *
import gaussian_kde

import warnings
warnings.filterwarnings('ignore')

import holoviews as hv
from holoviews import opts

encoding = "ISO-8859-1"

# you should clone this git to a subdirectory called 'BES_analysis_code' (in some directory - I call it BES_analysis - doesn't matter though)
# %matplotlib inline
(BES_code_folder, BES_small_data_files, BES_data_folder,
 BES_output_folder, BES_file_manifest, BES_R_data_files) = setup_directories()

global BES_Panel, BES_numeric, BES_reduced, BES_reduced_with_na, BES_non_numeric
data_subfolder = BES_data_folder + dataset_name + os.sep

(manifest, dataset_filename, dataset_description, dataset_citation,
 dataset_start, dataset_stop, dataset_wave) = get_manifest(dataset_name, BES_file_manifest)

for df in df_list:
    if df=="BES_Panel":
        globals()[df]  = pd.read_pickle(data_subfolder + dataset_filename.replace('.dta','.zip'),compression='zip')
    else:
        globals()[df]  = pd.read_pickle(data_subfolder + df + '.zip',compression='zip' )
        globals()[df].replace(-1,np.nan,inplace=True)
  
(var_type, cat_dictionary, new_old_col_names, old_new_col_names) = get_small_files(data_subfolder, encoding)

# get full set of inferred "cross wave" auth-lib/left-right values and ages
pan_dataset_allr_values = pd.read_csv(BES_small_data_files + "pan_dataset_allr_values"+".csv")
pan_dataset_ages = pd.read_csv( BES_small_data_files + "pan_dataset_ages"+".csv" )

Toggle code

var_type (7911, 14)


In [6]:
immigSelfW10 = search(BES_reduced_with_na,"immigSelfW10").index[0]
immigSelfW8  = search(BES_reduced_with_na,"immigSelfW8" ).index[0]
D8_10immigSelf = BES_reduced_with_na[immigSelfW10]-BES_reduced_with_na[immigSelfW8]

changeImmigW8  = search(BES_reduced_with_na,"changeImmigW8" ).index[0]
changeImmigW10 = search(BES_reduced_with_na,"changeImmigW10").index[0]
D8_10changeImmig = BES_reduced_with_na[changeImmigW10]-BES_reduced_with_na[changeImmigW8]

changeCrimeW8  = search(BES_reduced_with_na,"changeCrimeW8" ).index[0]
changeCrimeW10 = search(BES_reduced_with_na,"changeCrimeW10").index[0]
D8_10changeCrime = BES_reduced_with_na[changeCrimeW10]-BES_reduced_with_na[changeCrimeW8]

immigEconW10 = search(BES_reduced_with_na,"immigEconW10").index[0]
immigEconW8  = search(BES_reduced_with_na,"immigEconW8" ).index[0]
D8_10immigEcon = BES_reduced_with_na[immigEconW10]-BES_reduced_with_na[immigEconW8]

immigCulturalW10 = search(BES_reduced_with_na,"immigCulturalW10").index[0]
immigCulturalW8  = search(BES_reduced_with_na,"immigCulturalW8" ).index[0]
D8_10immigCultural = BES_reduced_with_na[immigCulturalW10]-BES_reduced_with_na[immigCulturalW8]


# df = pd.concat([D8_10immigSelf,D8_10changeImmig,D8_10changeCrime,D8_10immigEcon,D8_10immigCultural],axis=1).dropna()
# df.columns = ["D8_10immigSelf","D8_10changeImmig","D8_10changeCrime","D8_10immigEcon","D8_10immigCultural"]

In [23]:
from linearmodels.iv import IV2SLS
from linearmodels.iv import compare
import statsmodels.api as sm
from statsmodels.api import add_constant

In [26]:
df = pd.concat([D8_10immigSelf,D8_10changeImmig,D8_10changeCrime,D8_10immigEcon,D8_10immigCultural],axis=1).dropna().astype('float32')
df.columns = ["D8_10immigSelf","D8_10changeImmig","D8_10changeCrime","D8_10immigEcon","D8_10immigCultural"]
df = add_constant(df, has_constant='add')

In [38]:
# 2 stage least squares resolves to OLS when Endongeous and Instrument arguments are set to None
dep = "D8_10immigSelf"
exog = ['const','D8_10changeImmig']

res_ols = IV2SLS(dependent = df[dep],
                 exog = df[exog],
                 endog = None,
                 instruments = None).fit(cov_type='unadjusted')
print(res_ols)

# there's an effect of DchangeImmig in DimmigSelf ... but it's quite a small contribution

                            OLS Estimation Summary                            
Dep. Variable:         D8_10immigSelf   R-squared:                      0.0065
Estimator:                        OLS   Adj. R-squared:                 0.0064
No. Observations:               14901   F-statistic:                    97.465
Date:                Wed, Apr 15 2020   P-value (F-stat)                0.0000
Time:                        19:40:33   Distribution:                  chi2(1)
Cov. Estimator:            unadjusted                                         
                                                                              
                                Parameter Estimates                                 
                  Parameter  Std. Err.     T-stat    P-value    Lower CI    Upper CI
------------------------------------------------------------------------------------
const                0.0391     0.0167     2.3442     0.0191      0.0064      0.0719
D8_10changeImmig    -0.1551 

In [39]:
# First stage to check whether DchangeCrime is a strong instrument for changeImmig

dep = "D8_10changeImmig"
exog = ['const','D8_10changeCrime']

res_first = IV2SLS(dependent = df[dep],
                   exog = df[exog],
                   endog = None,
                   instruments = None).fit(cov_type='unadjusted')
print(res_first)

# correlation isn't huge, but F-statistic is pretty reasonable (F >> 10)

                            OLS Estimation Summary                            
Dep. Variable:       D8_10changeImmig   R-squared:                      0.0839
Estimator:                        OLS   Adj. R-squared:                 0.0838
No. Observations:               14901   F-statistic:                    1364.0
Date:                Wed, Apr 15 2020   P-value (F-stat)                0.0000
Time:                        19:40:35   Distribution:                  chi2(1)
Cov. Estimator:            unadjusted                                         
                                                                              
                                Parameter Estimates                                 
                  Parameter  Std. Err.     T-stat    P-value    Lower CI    Upper CI
------------------------------------------------------------------------------------
const               -0.4810     0.0074    -64.690     0.0000     -0.4956     -0.4664
D8_10changeCrime     0.3109 

In [40]:
# The second stage uses the instrument to fit the model.

dep = 'D8_10immigSelf'
exog = ['const']
endog = ['D8_10changeImmig']
instr = ['D8_10changeCrime']

res_second = IV2SLS(dependent = df[dep],
                    exog = df[exog],
                    endog = df[endog],
                    instruments = df[instr]).fit(cov_type='unadjusted')

print(res_second)
# DchangeImmig coefficienct is reduced to ~80% of previous

                          IV-2SLS Estimation Summary                          
Dep. Variable:         D8_10immigSelf   R-squared:                      0.0063
Estimator:                    IV-2SLS   Adj. R-squared:                 0.0062
No. Observations:               14901   F-statistic:                    5.5622
Date:                Wed, Apr 15 2020   P-value (F-stat)                0.0184
Time:                        19:40:35   Distribution:                  chi2(1)
Cov. Estimator:            unadjusted                                         
                                                                              
                                Parameter Estimates                                 
                  Parameter  Std. Err.     T-stat    P-value    Lower CI    Upper CI
------------------------------------------------------------------------------------
const                0.0521     0.0300     1.7397     0.0819     -0.0066      0.1109
D8_10changeImmig    -0.1280 

In [41]:
# Sanity check with 'Direct' calculated from the difference between DchangeImmig and residuals from first stage

dep = 'D8_10immigSelf'
df['D8_10changeImmig_hat'] = df["D8_10changeImmig"] - res_first.resids
exog = ['const','D8_10changeImmig_hat']

res_direct = IV2SLS(dependent = df[dep],
                    exog = df[exog],
                    endog = None,
                    instruments = None).fit(cov_type='unadjusted')

print(compare({'OLS':res_ols, '2SLS':res_second, 'Direct': res_direct}))

# The 2SLS coefficient on D8_10changeImmig and the direct coefficient on D8_10changeImmig_hat are identical.

                                 Model Comparison                                
                                        OLS               2SLS             Direct
---------------------------------------------------------------------------------
Dep. Variable                D8_10immigSelf     D8_10immigSelf     D8_10immigSelf
Estimator                               OLS            IV-2SLS                OLS
No. Observations                      14901              14901              14901
Cov. Est.                        unadjusted         unadjusted         unadjusted
R-squared                            0.0065             0.0063             0.0004
Adj. R-squared                       0.0064             0.0062             0.0003
F-statistic                          97.465             5.5622             5.5292
P-value (F-stat)                     0.0000             0.0184             0.0187
======================     ================   ================   ================
const           

In [25]:
# If we had multiple exogenous variables you couldn't just rely on "big F in first stage" => strong first stage
# there is a helper function that calculation partial F (and a bunch of other partial measures) for the instrument
# upon your candidate endogenous value

print(res_second.first_stage)

# surprisingly memory intensive!

       First Stage Estimation Results      
                           D8_10changeImmig
-------------------------------------------
R-squared                            0.0839
Partial R-squared                    0.0839
Shea's R-squared                     0.0839
Partial F-statistic                  631.44
P-value (Partial F-stat)             0.0000
Partial F-stat Distn                chi2(1)
==========================      ===========
const                               -0.4810
                                  (-64.717)
D8_10changeCrime                     0.3109
                                   (25.129)
-------------------------------------------

T-stats reported in parentheses
T-stats use same covariance type as original model


In [29]:
# Can also specify the model with a formula

formula = ('D8_10immigSelf ~ 1 + [D8_10changeImmig ~ D8_10changeCrime]')
mod = IV2SLS.from_formula(formula, df)
res_formula = mod.fit(cov_type='unadjusted')

print(compare({'OLS':res_ols, '2SLS':res_second, 'Direct': res_direct, 'From Formula':res_formula}))

# Changes nothing other than that 'const' is renamed 'Intercept'

                                          Model Comparison                                          
                                        OLS               2SLS             Direct       From Formula
----------------------------------------------------------------------------------------------------
Dep. Variable                D8_10immigSelf     D8_10immigSelf     D8_10immigSelf     D8_10immigSelf
Estimator                               OLS            IV-2SLS                OLS            IV-2SLS
No. Observations                      14901              14901              14901              14901
Cov. Est.                        unadjusted         unadjusted         unadjusted         unadjusted
R-squared                            0.0065             0.0063             0.0004             0.0063
Adj. R-squared                       0.0064             0.0062             0.0003             0.0062
F-statistic                          97.465             5.5622             5.5292          

In [30]:
# It also handles categorical variables - adding controlImmigW9 as a categorical
# it drops the first category by default, so I'm setting the order so that the first category is the highest frequency
# "Some Control" is thus the base (shame it doesn't show that automatically - boo!)

df["controlImmigW9"] = BES_Panel["controlImmigW9"]
df["controlImmigW9"] = df["controlImmigW9"].cat.reorder_categories( list(BES_Panel["controlImmigW9"].value_counts().index) )

dep = 'D8_10immigSelf'
exog = ['const','controlImmigW9']
endog = ['D8_10changeImmig']
instr = ['D8_10changeCrime']

res_cat = IV2SLS(   dependent = df[dep],
                    exog = df[exog],
                    endog = df[endog],
                    instruments = df[instr]).fit(cov_type='unadjusted')

print(res_cat)

                          IV-2SLS Estimation Summary                          
Dep. Variable:         D8_10immigSelf   R-squared:                      0.0075
Estimator:                    IV-2SLS   Adj. R-squared:                 0.0071
No. Observations:               14901   F-statistic:                    19.276
Date:                Wed, Apr 15 2020   P-value (F-stat)                0.0037
Time:                        18:42:18   Distribution:                  chi2(6)
Cov. Estimator:            unadjusted                                         
                                                                              
                                        Parameter Estimates                                         
                                  Parameter  Std. Err.     T-stat    P-value    Lower CI    Upper CI
----------------------------------------------------------------------------------------------------
const                                0.0709     0.0381     1.8586

In [33]:
## Post-estimation diagnostics

In [42]:
# Is changeImmig actually endogeneous to immigSelf?

res_second.wooldridge_regression

## Nope

Wooldridge's regression test of exogeneity
H0: Endogenous variables are exogenous
Statistic: 0.2732
P-value: 0.6012
Distributed: chi2(1)
WaldTestStatistic, id: 0xe9d53748

In [ ]:
# How much do I buy this as a measure of endogeneity? Quick sanity check - will test on "really obviously endogeneous" variables

In [44]:
# Add in DimmigEcon, DimmigCultural

df = pd.concat([D8_10immigSelf,D8_10changeImmig,D8_10changeCrime,D8_10immigEcon,D8_10immigCultural],axis=1).dropna()
df.columns = ["D8_10immigSelf","D8_10changeImmig","D8_10changeCrime","D8_10immigEcon","D8_10immigCultural"]
df = add_constant(df, has_constant='add')

In [46]:
dep = 'D8_10immigSelf'
exog = ['const']
endog = ['D8_10immigEcon']
instr = ['D8_10changeCrime',"D8_10changeImmig","D8_10immigCultural"] ## these are not necessarily effective/sensible instruments!

res_test = IV2SLS(  dependent = df[dep],
                    exog = df[exog],
                    endog = df[endog],
                    instruments = df[instr]).fit(cov_type='unadjusted')
print(res_test)

# F value looks bonkers, R-squared seems very low

                          IV-2SLS Estimation Summary                          
Dep. Variable:         D8_10immigSelf   R-squared:                     -0.0565
Estimator:                    IV-2SLS   Adj. R-squared:                -0.0566
No. Observations:               14901   F-statistic:                    467.92
Date:                Wed, Apr 15 2020   P-value (F-stat)                0.0000
Time:                        19:44:29   Distribution:                  chi2(1)
Cov. Estimator:            unadjusted                                         
                                                                              
                               Parameter Estimates                                
                Parameter  Std. Err.     T-stat    P-value    Lower CI    Upper CI
----------------------------------------------------------------------------------
const             -0.1775     0.0204    -8.6921     0.0000     -0.2175     -0.1375
D8_10immigEcon     0.6739     0.0312

In [47]:
# Is DimmigEcon endogeneous to DimmigSelf?

res_test.wooldridge_regression

# Yes, very much so

Wooldridge's regression test of exogeneity
H0: Endogenous variables are exogenous
Statistic: 239.5133
P-value: 0.0000
Distributed: chi2(1)
WaldTestStatistic, id: 0x71911f48

In [48]:
# How about those instruments? Too many or just enough?

res_test.wooldridge_overid

# Hella too many (/too correlated with the dependent variable)

Wooldridge's score test of overidentification
H0: Model is not overidentified.
Statistic: 14.4561
P-value: 0.0007
Distributed: chi2(2)
WaldTestStatistic, id: 0xe9d53548

In [49]:
# Lets drop DimmigCultural as an instrument

dep = 'D8_10immigSelf'
exog = ['const']
endog = ['D8_10immigEcon']
instr = ['D8_10changeCrime',"D8_10changeImmig"] 

res_test2 = IV2SLS(  dependent = df[dep],
                    exog = df[exog],
                    endog = df[endog],
                    instruments = df[instr]).fit(cov_type='unadjusted')
print(res_test2)

                          IV-2SLS Estimation Summary                          
Dep. Variable:         D8_10immigSelf   R-squared:                     -0.3980
Estimator:                    IV-2SLS   Adj. R-squared:                -0.3981
No. Observations:               14901   F-statistic:                    69.429
Date:                Wed, Apr 15 2020   P-value (F-stat)                0.0000
Time:                        19:49:38   Distribution:                  chi2(1)
Cov. Estimator:            unadjusted                                         
                                                                              
                               Parameter Estimates                                
                Parameter  Std. Err.     T-stat    P-value    Lower CI    Upper CI
----------------------------------------------------------------------------------
const             -0.3992     0.0640    -6.2360     0.0000     -0.5247     -0.2738
D8_10immigEcon     1.1875     0.1425

In [51]:
res_test2.wooldridge_overid

# Bingo - now it's not overidentified
# Ironically, changeImmig is sufficiently exogeneous to immigSelf to be an instrument

Wooldridge's score test of overidentification
H0: Model is not overidentified.
Statistic: 0.0207
P-value: 0.8856
Distributed: chi2(1)
WaldTestStatistic, id: 0x71820088

In [52]:
# But it's not clear whether DchangeCrime/DchangeImmig fulfil the first criterion of 'instrumentality'

print(res_test2.first_stage)

      First Stage Estimation Results     
                           D8_10immigEcon
-----------------------------------------
R-squared                          0.0094
Partial R-squared                  0.0094
Shea's R-squared                   0.0094
Partial F-statistic                141.95
P-value (Partial F-stat)           0.0000
Partial F-stat Distn              chi2(2)
==========================    ===========
const                              0.3685
                                 (31.274)
D8_10changeCrime                   0.0049
                                 (0.3953)
D8_10changeImmig                  -0.1320
                                (-11.512)
-----------------------------------------

T-stats reported in parentheses
T-stats use same covariance type as original model


In [ ]:
# alternative endogeneity tests

In [59]:
res_test.durbin()
# Not robust to heteroscedasity (unlike wooldridge)

Durbin test of exogeneity
H0: All endogenous variables are exogenous
Statistic: 239.0375
P-value: 0.0000
Distributed: chi2(1)
WaldTestStatistic, id: 0xeb1c3a08

In [60]:
res_test.wu_hausman()
# Slightly modified version of Durbin
# Also, not robust to heteroscedasity (unlike wooldridge)

Wu-Hausman test of exogeneity
H0: All endogenous variables are exogenous
Statistic: 242.8856
P-value: 0.0000
Distributed: F(1,14898)
WaldTestStatistic, id: 0xec89d888

In [61]:
# Robust to heteroscedasity
# but less power
res_test.wooldridge_score

Wooldridge's score test of exogeneity
H0: Endogenous variables are exogenous
Statistic: 162.0395
P-value: 0.0000
Distributed: chi2(1)
WaldTestStatistic, id: 0x717c9908

In [53]:
# Better covariance models? Yes - several are accessible - but the default one is "robust" (to heteroscedasticity)

dep = 'D8_10immigSelf'
exog = ['const']
endog = ['D8_10immigEcon']
instr = ['D8_10changeCrime',"D8_10changeImmig"] 

res_test3 = IV2SLS(  dependent = df[dep],
                    exog = df[exog],
                    endog = df[endog],
                    instruments = df[instr]).fit()
print(res_test3)

# Coefficients don't change, just the F-statistic/comparable error/T-stats get a bit more (accurately) conservative

                          IV-2SLS Estimation Summary                          
Dep. Variable:         D8_10immigSelf   R-squared:                     -0.3980
Estimator:                    IV-2SLS   Adj. R-squared:                -0.3981
No. Observations:               14901   F-statistic:                    54.874
Date:                Wed, Apr 15 2020   P-value (F-stat)                0.0000
Time:                        19:59:38   Distribution:                  chi2(1)
Cov. Estimator:                robust                                         
                                                                              
                               Parameter Estimates                                
                Parameter  Std. Err.     T-stat    P-value    Lower CI    Upper CI
----------------------------------------------------------------------------------
const             -0.3992     0.0703    -5.6775     0.0000     -0.5371     -0.2614
D8_10immigEcon     1.1875     0.1603